# Preprocessing

In [2]:
# Unzip files in their respective folders
from scripts.unzip_files import unzip_files_KGS 

source_folder = '../data/zip_files'
destination_folder = '../data/v1.0_raw_data'

unzip_files_KGS(source_folder, destination_folder)

In [1]:
# Download las files from KGS
from scripts.download_las_files import download_and_organize_las_files

source_folder = '../data/v1.0_raw_data'  
destination_folder = '../data/v2.0_zip_files'

# File to map las file names to urls
database_path = '../data/zip_files/ks_las_files.txt'  

# Download and organize las files
download_and_organize_las_files(source_folder, destination_folder, database_path)

JonesCanyonSoutheast |                                                  
Schaben              |                                                  
Marcotte             |                                                  
Cutter               |                                                  
MedicineLodgeNorth   |                                                  
KraftPrusa           |                                                  
Wellington           |                                                  
Burrton              |                                                  
GarnettShoestring    |                                                  
BemisShutts          |                                                  
BradshawGasArea      |                                                  
Webster              |                                                  
Morel                |                                                  
ElDorado             |                             

In [1]:
from scripts.ajust_las_files import process_las_files

source_folder = '../data/v2.0_zip_files'
destination_folder = '../data/v3.0_las_files'
csv_folder = '../data/v1.0_raw_data'

process_las_files(source_folder, destination_folder, csv_folder)

Arroyo               |                                                  
BemisShutts          |                                                  
BradshawGasArea      |                                                  
Burrton              |                                                  
Cooper               |                                                  
EastFork             |                                                  
Cutter               |                                                  
ElDorado             |                                                  
GarnettShoestring    |                                                  
HallGurney           |                                                  
JonesCanyonSoutheast |                                                  
KraftPrusa           |                                                  
Marcotte             |                                                  
MedicineLodgeNorth   |                             

### PENDIENTES:

LLM with RAG to check formation

In [3]:
%load_ext autoreload
%autoreload 2

from scripts.widgets import create_directory_selector
from scripts.exploratory_data_analysis import load_las_files_from_field
import ipywidgets as widgets
from IPython.display import display
import os

base_data_path = '../data/v3.0_las_files'

# Crea el selector de directorios
source_directory_selector = create_directory_selector('Select Field:', base_data_path)

# Output para mostrar resultados
output = widgets.Output()

project = None

def on_source_directory_selected(change):
    # Indicar que queremos utilizar la variable global 'project'
    global project
    with output:
        # output.clear_output()
        selected_directory = change['new']
        
        # Aquí asumimos que 'load_las_files_from_field' es una función que sabes cómo implementar
        project = load_las_files_from_field(os.path.join(base_data_path, selected_directory))


# Observa cambios en el selector de directorios
source_directory_selector.observe(on_source_directory_selected, names='value')

# Muestra los widgets
display(source_directory_selector, output)


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Dropdown(description='Select Field:', options=('JonesCanyonSoutheast', 'Schaben', 'Marcotte', 'Cutter', 'Medic…

Output()

In [8]:
project

Index,UWI,Data,Curves
0,,14 curves,"BVTX, AVTX, SWNC, SCAL, SWPOR, SP, LGRD, GR, MCAL, MI, MN, DT, ITT, SPOR"
1,Culwell 'JT' #1-11_2,20 curves,"AVTX, BVTX, CILD, CNLS, DCAL, DPOR, DT, GR, ITT, MCAL, MI, MN, RHOB, RHOC, RILD, RILM, RLL3, RXORT, SP, SPOR"
2,,20 curves,"BVTX, AVTX, RXORT, CILD, RLL3, SP, RILD, RILM, DCAL, RHOB, RHOC, DPOR, CNLS, GR, MCAL, MI, MN, DT, ITT, SPOR"
3,Patti #1-10_2,20 curves,"AVTX, BVTX, CILD, CNLS, DCAL, DPOR, DT, GR, ITT, MCAL, MI, MN, RHOB, RHOC, RILD, RILM, RLL3, RXORT, SP, SPOR"
4,,20 curves,"BVTX, AVTX, RXORT, CILD, RLL3, SP, RILD, RILM, DCAL, RHOB, RHOC, DPOR, CNLS, GR, MCAL, MI, MN, DT, ITT, SPOR"
5,Theron #1-10_7,20 curves,"AVTX, BVTX, CILD, CNLS, DCAL, DPOR, DT, GR, ITT, MCAL, MI, MN, RHOB, RHOC, RILD, RILM, RLL3, RXORT, SP, SPOR"
6,Culwell C # 2-10_3,20 curves,"AVTX, BVTX, CILD, CNLS, DCAL, DPOR, DT, GR, ITT, MCAL, MI, MN, RHOB, RHOC, RILD, RILM, RLL3, RXORT, SP, SPOR"
7,,14 curves,"BVTX, AVTX, SWNC, SCAL, SWPOR, SWLS, SWSS, SWDL, SP, LGRD, GR, MCAL, MI, MN"
8,"Culwell ""B"" #1-11_1",20 curves,"AVTX, BVTX, CILD, CNLS, DCAL, DPOR, DT, GR, ITT, MCAL, MI, MN, RHOB, RHOC, RILD, RILM, RLL3, RXORT, SP, SPOR"
9,,22 curves,"BVTX, AVTX, RXORT, CILD, RLL3, SP, RILD, RILM, DCAL, RHOB, RHOC, DPOR, CNLS, CNSS, CNDL, GR, MCAL, MI, MN, DT, ITT, SPOR"


In [ ]:
source = '../data/v2.0_zip_files'

# write a code to count all .las files in source
